In [1]:
import json
import re
from pymatgen.core import Composition, Element
import pprint as pp
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle as pkl
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import sys, os
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
sys.path.append('/home/jupyter/CJK/TempTime')
from data_extraction_rxn_classification_all_targets import *
from sklearn.dummy import DummyRegressor

In [2]:
all_elements = [Element.from_Z(i).symbol for i in range(1, 119)]

In [3]:
with open('/home/jupyter/CJK/TempTime/data/solid-state_dataset_20200713.json') as f:
    ss_data= json.load(f)
ss_reactions = ss_data["reactions"]
ss_extracted, ss_precursor_nomenclature = extract_solidstate(ss_reactions, max_pre=5)

Returning extracted data of 51574/31782 reactions.


In [4]:
with open('/home/jupyter/CJK/TempTime/data/ss_extracted_NO_IMPUTATION_precs_all_targets.pkl', 'rb') as f:
    papers = pkl.load(f)

In [5]:
papers = [x for x in papers if not np.isnan(x['temp_time_vector'][4])]

In [6]:
papers

[{'DOI': '10.1149/1.3236634',
  'target': 'Ba1.95MgEu0.05B2O6',
  'precursors': ['B(HO)3', 'Ba(NO3)2', 'Eu2O3', 'Mg(NO3)2'],
  'operation_types': ['StartingSynthesis',
   'MixingOperation',
   'MixingOperation',
   'MixingOperation',
   'MixingOperation',
   'MixingOperation',
   'MixingOperation',
   'MixingOperation',
   'DryingOperation',
   'HeatingOperation',
   'HeatingOperation',
   'HeatingOperation',
   'StartingSynthesis'],
  'operation_tokens': ['prepared',
   'dissolved',
   'dissolved',
   'stirring',
   'mixed',
   'stirred',
   'stirred',
   'adjusted',
   'drying',
   'calcinated',
   'calcinated',
   'firing',
   'prepared'],
  'operation_times': [None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   12,
   None,
   None,
   None],
  'operation_temps': [None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   600.0,
   None,
   None,
   None],
  'temp_time_vector': array([600.,  nan,  nan,  nan,  12.,  nan,  nan,  nan]

In [7]:
targets = [x['target'] for x in papers]
precursors = [x['precursors'] for x in papers]
temps = [x['temp_time_vector'][4] for x in papers]
dois = [x['DOI'] for x in papers]

In [8]:
# get means
only_ss_rxns = []
for target, precs, temps, doi in zip(targets, precursors, temps, dois):
    if len(precs) == 0:
        continue
    found = False
    for result in only_ss_rxns:
        if result["target"] == target and set(result["precursors"]) == set(precs):
            result["ss_sinter_temps"].append(temps)
            result["DOIs"].append(doi)
            found = True
    if not found:
        new_result = {}
        new_result["target"] = target
        new_result["precursors"] = precs
        new_result["ss_sinter_temps"] = [temps]
        new_result["DOIs"] = [doi]
        only_ss_rxns.append(new_result)

In [9]:
times = [np.mean(x['ss_sinter_temps']) for x in only_ss_rxns]
X = np.zeros(len(times))
times = np.array(times)
X = np.array(X)

In [10]:
X, times = shuffle(X, times, random_state=42)

In [12]:
kf = KFold(n_splits=10, shuffle=False)
maes = []
rmses = []
r2s = []
mres = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = times[train_index], times[test_index]
    dummy_regr = DummyRegressor(strategy="mean")
    dummy_regr.fit(X_train, y_train)
    preds = dummy_regr.predict(X_test)
    

    mae = mean_absolute_error(y_test, preds)
    rmse = mean_squared_error(y_test, preds, squared=False)
    r2 = r2_score(y_test, preds)
    mre = np.mean((np.abs(y_test-preds)/y_test)*100)
    
    maes.append(mae)
    rmses.append(rmse)
    r2s.append(r2)
    mres.append(mre)
    

In [13]:
print(np.mean(maes), np.std(maes))
print(np.mean(rmses), np.std(rmses))
print(np.mean(r2s), np.std(r2s))
print(np.mean(mres), np.std(mres))

5.4382322670439835 0.16028517719806473
7.008743781341375 0.2543667512207952
-0.001676202733917398 0.0015588718626602799
117.07800460666917 5.084195139080903


In [15]:
len(ss_extracted)

51574